
# INTRODUCCIÓN
---

**MODELOS DE LENGUAJE Y PREDICCIÓN DE PALABRAS**


*Tipos de modelos de lenguaje*: Existen principalmente dos tipos de modelos de
lenguaje:

*   *Modelos de lenguaje estadístico:* estos modelos utilizan técnicas estadísticas tradicionales como N-gramas, modelos ocultos de Markov (HMM) y ciertas reglas lingüísticas para aprender la distribución de probabilidad de las palabras

*   *Modelos de lenguaje neuronal:* utilizan diferentes tipos de redes neuronales para modelar el lenguaje.

En este cuaderno abordaremos los modelos de lenguaje estadístico.

**INFERENCIA ESTADÍSTICA: modelos de n-gramas**

La inferencia estadística en general consiste en tomar algunos datos, generados según alguna distribución de probabilidad desconocida, y hacer algunas inferencias sobre esta distribución.


## MODELADO DE LENGUAJE

---

Un modelo de lenguaje aprende a predecir la probabilidad de una secuencia de palabras dado un conjunto de palabras previas.

Esta tarea es fundamental para reconocimiento de voz, reconocimiento óptico de caracteres, corrección ortográfica, traducción automática, entre otras.

*   **PREDICCIÓN DE LA SIGUIENTE PALABRA:**
Lo que se intenta es calcular la función de probabilidad $P$

$$P(W_n / W_1, \ldots, W_{n-1})$$

Por ejemplo, en un modelo de bigramas buscaremos predecir la siguiente palabra a partir de la función de probabilidades de exactamente la palabra anterior.

Se utiliza la clasificaicón de las palabras previas, aunque debe considerarse que existen secuencias de palabras que no se encuentren en el historial, para obtener predicciones razonables dado este problema se agrupan segmentos históricos similares, utilizando la propiedad de Markov, que prioriza contextos locales.

Los modelos basados en n-gramas utilizan secuencias de 2,3,4  palabras consecutivas.

*   **ENFOQUE DE MAXIMA VEROSIMILITUD**

Para obtener las probabilidade que nos permitan estimar la probabilidad de aparición de una parabra objetivos, es necesario calcular

$$P(W_n | W_1, \ldots, W_{n-1}) = \frac{C(W_1, \ldots, W_{n-1}, W_n)}{C(W_1, \ldots, W_{n-1})}$$

Donde C(⋅) es el conteo de las secuencias de palabras de longitud $n$, por ejemplo, $n=2$ (dos palabras), $n=3$ (tres palabras), así sucesivamente.

Para calcular la probabilidad de una palabra $y$ dada una palabra previa $x$, se calcula el conteo de los bigramas  $C(xy)$ y se normaliza con todos los bigramas que comparten la primera palabra $x$, que es lo mismo que los unigramas de $x$


*   **SENSIBILIDAD DE LOS MODELOS DE N-GRAMAS Y SUAVIZADO**

Como otros modelos estadísticos, el modelo de n-gramas depende en gran medida  del conjunto de datos de entrenamiento.

Lo que implica que las probabilidades codifican cuestiones específicas del conjunto de datos.

Para atenuar el problema de enfrentarse a probabilidades cero de n-gramas, se usa un suavizado de las probabilidades del conjunto de datos, esto es, se corta un poco de la masa de probabilidades de los conteos mayores, y se traslada a las secuencias que no tienen conteos.

**Suavizado de Laplace**

Suavizado de Laplace o ley de Laplace es una técnica sencilla que consiste en proporcionar un poco del espacio de probabilidades a los eventos no vistos.

Esto es, se usa la matriz de conteo, por ejemplo de bigramas, y se suma 1 a todos los conteos, para posteriormente normalizar en probabilidades, bigramas que no ocurrieron en el conjunto de datos al menos tendrán una ocurrencia usando suavizado de Laplace.

La probabilidad ajustada sería como sigue, donde se requiere ajustar agregando el tamaño del vocabulario V.
_________________________

In [3]:
# Dependencies
import re
import os
import sys
import io
import pandas as pd
import zipfile
import json
import numpy as np
import ast
from sklearn.decomposition import PCA
import plotly.express as px
#
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Dataset Original:
# https://www.kaggle.com/datasets/suchintikasarkar/sentiment-analysis-for-mental-health/data?select=Combined+Data.csv

# Usaremos solo el 5% del dataset, la selección fue leatoria con:
# df_combined_sample = df_combined.sample(frac=0.05, random_state=42)

In [5]:
# Path to the zip file
zip_file_path = 'Data_embedding.csv.zip'
# Name of the CSV file inside the zip
csv_file_name = 'Data_embedding.csv'

# Extract the CSV file from the zip archive
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extract(csv_file_name, path='.') # Extract to current directory

# Load the data from Data_embedding.csv into a pandas DataFrame
df = pd.read_csv(csv_file_name)
df.head()

,statement,status,embedding
0,Just as the the title says. I feel like one is...,Depression,"[2.7965152185061015e-05, -0.020042788237333298..."
1,a blackened sky encroached tugging behind it m...,Depression,"[0.013387450017035007, -0.03580772131681442, 0..."
2,"It gives you insomnia, which in turn makes you...",Depression,"[-0.017405608668923378, -0.01630398817360401, ..."
3,"Hello all, I'm a new submitter to this channel...",Normal,"[0.013213454745709896, -0.004268334247171879, ..."
4,Thank God the CB is over for Eid,Normal,"[0.07578383386135101, -0.019401390105485916, -..."


In [6]:
# Display the frequency table of the 'status' column
status_counts = df['status'].value_counts()
display(status_counts)

,count
status,
Normal,831
Depression,814
Suicidal,484
Anxiety,186
Bipolar,137
Stress,134
Personality disorder,66


In [7]:
# --- 1) Preparar X (matriz de embeddings) ---
cols_req = ["embedding", "status", "statement"]
mask = df[cols_req].notna().all(axis=1)
df_ = df.loc[mask, cols_req].copy()


In [8]:
# Convierte la columna embedding (lista/array por fila en formato string) en una matriz 2D (n_muestras x dim)
# Usa ast.literal_eval para convertir las strings de lista en listas de Python
X = np.array([ast.literal_eval(e) for e in df_["embedding"].tolist()], dtype=float)

#assert X.ndim == 2, "Cada 'embedding' debe ser una lista/array de igual longitud."
#assert np.isfinite(X).all(), "Hay valores no finitos (NaN/Inf) en los embeddings."

In [9]:
# --- 2) PCA a 3 componentes ---
# Nota: PCA centra automáticamente las features (no escala a var=1).
pca = PCA(n_components=3, random_state=0)
X_pca = pca.fit_transform(X)  # shape: (n_muestras, 3)


In [10]:
# --- 3) DataFrame para graficar ---
df_plot = pd.DataFrame(
    X_pca, columns=["PC1", "PC2", "PC3"], index=df_.index
).assign(
    status=df_["status"].astype(str),
    statement=df_["statement"].astype(str)
)

In [11]:
# Títulos con varianza explicada
var = pca.explained_variance_ratio_
axis_titles = {
    "x": f"PC1 ({var[0]:.1%})",
    "y": f"PC2 ({var[1]:.1%})",
    "z": f"PC3 ({var[2]:.1%})",
}
axis_titles

{'x': 'PC1 (7.6%)', 'y': 'PC2 (3.2%)', 'z': 'PC3 (1.9%)'}

In [12]:
# --- 4) Gráfica 3D interactiva con Plotly ---
fig = px.scatter_3d(
    df_plot, x="PC1", y="PC2", z="PC3",
    color="status",
    hover_data={"statement": True, "status": True, "PC1": ':.3f', "PC2": ':.3f', "PC3": ':.3f'},
    opacity=0.85,
    width=900, height=650,
    title="PCA de embeddings (3D)"
)

fig.update_layout(
    scene=dict(
        xaxis_title=axis_titles["x"],
        yaxis_title=axis_titles["y"],
        zaxis_title=axis_titles["z"],
    ),
    legend_title_text="status",
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.show()